# 스테그아날리시스 데이터를 이용한 파이토치 분류 가이드

파이토치 설치 -> 윈도우즈는 아나콘다 필수!<br>
https://pytorch.org/get-started/locally/<br>
conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch

## CNN 네트워크 구성

nn.Squential을 이용해 한 파일에 구현할 수 있지만, 보통 클래스 형태로 따로 사용.

In [5]:
# -*- coding: utf-8 -*-
import torch
from torch import nn

class Model(nn.Module):
    def __init__(self, classes=2):
        super(Model, self).__init__()
        #Group1
        self.conv1 = nn.Conv2d(1,8,5,1,2, bias=False)
        self.bn1 = nn.BatchNorm2d(8)
        
        #Group2
        self.conv2 = nn.Conv2d(8,16,5,1,2)
        self.bn2 = nn.BatchNorm2d(16)
        
        #Group 3~5
        self.conlayer3 = self.Conv_forward(16,32,5,1,2)
        self.conlayer4 = self.Conv_forward(32,64,5,1,2)
        self.conlayer5 = self.Conv_forward(64,128,5,1,2)
        self.AvgPooling = nn.AvgPool2d(5,2,2)
        self.GlobalPooling = nn.AvgPool2d(16)
        
        #Fully Connected
        self.fc = nn.Sequential(
                nn.Linear(128,classes),
                nn.Softmax(dim=1)
                )
        
    def Conv_forward(self, in_featere, out_feature, k_size, stride, padding, bias=False):
        Seq = nn.Sequential(
                nn.Conv2d(in_featere, out_feature, k_size,stride,padding,bias=bias),
                nn.BatchNorm2d(out_feature),
                nn.ReLU(),
                )
        
        return Seq
    
    def forward(self, x):
        #Group1
        x = self.conv1(x)
        x = torch.abs(x)
        x = self.bn1(x)
        x = torch.tanh(x)
        x = self.AvgPooling(x)
        
        #Group2
        x = self.conv2(x)
        x = self.bn2(x)
        x = torch.tanh(x)
        x = self.AvgPooling(x)
        
        #Group3~5
        x = self.conlayer3(x)
        x = self.AvgPooling(x)
        x = self.conlayer4(x)
        x = self.AvgPooling(x)
        x = self.conlayer5(x)
        x = self.GlobalPooling(x)        
        
        #Flatten & FC layer
        x = x.view(-1,128)
        x = self.fc(x)
        return x
    
if __name__ == "__main__":
    x = torch.randn([1,1,256,256])
    net = Model()
    y = net(x)
    print(y)

tensor([[0.5406, 0.4594]], grad_fn=<SoftmaxBackward>)


## 데이터 구성
torch.utils.data의 TensorDataset, DataLoader 클래스를 사용한다.<br>
파이토치의 데이터는 [데이터수, 채널수, 너비, 높이]의 형태

In [26]:
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

#넘파이 데이터 256사이즈의 흑백영상 1000장 이라고 가정, 2진 분류
data = np.zeros([1000,1,256,256], dtype=np.uint8)
label = np.zeros([1000, 2], dtype=np.long)

#파이토치 텐서로 변환
data = torch.from_numpy(data).float()
label = torch.from_numpy(label).long()

ds = TensorDataset(data, label)
data_loader = DataLoader(ds, batch_size=100, shuffle=True) #셔플 true == 데이터를 알아서 섞어줌, 학습시에만 true

## 학습
파이토치의 학습은 직관적인 편이다.<br>

In [25]:
from torch import optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #gpu 사용 가능?

#네트워크 설정
Net = Model()
Net = Net.to(device)
lr = 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(Net.parameters(), lr=lr, momentum=0.9)

#데이터 로더에서 자동으로 배치사이즈씩 나누어서 데이터를 보내준다.
for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)
    
    #이전 단계의 기울기 초기화
    optimizer.zero_grad()
    
    #예측 [P(a), P(b)]
    y_pred = Net(X) 
    
    #loss 계산
    loss = loss_fn(y_pred, torch.max(Y, 1)[1])
    
    #학습
    loss.backward()
    optimizer.step()
print(X.shape, Y.shape, y_pred[0])

#파라미터 저장
#torch.save(Net.state_dict(), save_path + f"epoch{epo}.pth")

torch.Size([100, 1, 256, 256]) torch.Size([100, 2]) tensor([0.5051, 0.4949], device='cuda:0', grad_fn=<SelectBackward>)


## 검증

In [21]:
import tqdm

Net = Model()
Net = Net.to(device).eval()

#학습 파라미터 불러오기
#Net.load_state_dict(torch.load(save_path + "epoch100.prm"), strict=False)

ys = []
ypreds = []

#for epoch in range(100):
for X, Y in tqdm.tqdm(data_loader):
    X = X.to(device)
    Y = Y.to(device)
        
    with torch.no_grad():
        # Value, Indices >> Get Indices
        _, y_pred = Net(X).max(1)
        ys.append(Y.max(1)[1])
        ypreds.append(y_pred)
        
ys = torch.cat(ys)
ypreds = torch.cat(ypreds)
print("정확도:", ((ys == ypreds).float().sum() / 10000).item())

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 55.44it/s]

정확도: 0.09999999403953552
